### Checking Yahoo Finance for streamlit app

In [77]:
import pinecone
from langchain.vectorstores import FAISS,Pinecone

In [83]:
import yaml
import yahoo_fin.stock_info as si
import yfinance as yf
from langchain.llms import Cohere, OpenAI
from langchain.chains import SequentialChain,AnalyzeDocumentChain
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from langchain.chat_models import ChatAnthropic
from langchain.chains.summarize import load_summarize_chain
import os
from datetime import datetime

with open("../data/apis.yaml", "r") as file:
    yaml_data = yaml.load(file, Loader=yaml.FullLoader)
open_ai_params = {
    "max_tokens": 512,
    "openai_api_key": yaml_data["LLMS"]["OPENAI_API_KEY"],
}
cohere_params = {
    "model": "command-xlarge-nightly",
    "max_tokens": 2202,
    "cohere_api_key": yaml_data["LLMS"]["COHERE_API_KEY"],
    "temperature": 0,
    "k": 0,
}

claude_params = {'anthropic_api_key':yaml_data["LLMS"]["CLAUDE_API_KEY"],
                'model':'claude-instant-v1.1-100k','max_tokens_to_sample':30000}
pinecone.init(api_key= yaml_data['PINECONE']['API_KEY'],environment= yaml_data['PINECONE']['ENV'])
index_name = "financial-analysis"
oai = OpenAIEmbeddings(openai_api_key = open_ai_params['openai_api_key'])
pinecone_db = Pinecone.from_existing_index('financial-analysis', oai)
from llm import LLM_analysis

In [84]:
x = LLM_analysis('AAPL',open_ai_params,cohere_params,claude_params)

In [87]:
print(x.qachain_anthropic(pinecone_db,'''How are you?''')[0])

[*********************100%***********************]  1 of 1 completed
Use the following information to answer the question at the end in a coherent summary. 
    The below contains information about AAPL and you are a financial analyst
    Here are the summary of the latest SEC filings figures
Risk FactorsThe Company’ s business, reputation, results of operations, financial condition and stock price can be affected by a number of factors, whether currently known or unknown, including those described in Part I, Item 1A of the 2022 Form 10-K under the heading“ Risk Factors.”EuropeEurope net sales decreased during the first quarter of 2023 compared to the same quarter in 2022 due to the weakness in foreign currencies relative to the U. S. dollar, which contributed to lower net sales of iPhone and Mac.The weakness of the renminbi contributed to lower net sales of iPhone, which was partially offset by higher net sales of iPad. JapanJapan net sales decreased during the first quarter of 2023 c

In [88]:
from openbb_terminal.sdk import openbb

In [104]:
openbb.stocks.ca.balance(['AAPL','TSLA'])

Other available yearly timeframes are: 2018, 2019, 2020, 2021, 2022



,AAPL,TSLA
Item,,
Cash & Short Term Investments,48.3B,22.48B
Cash & Short Term Investments Growth,-22.89%,24.52%
Cash Only,21.48B,16.55B
Short-Term Investments,-,-
Cash & ST Investments / Total Assets,13.69%,27.30%
Total Accounts Receivable,60.93B,2.95B
Total Accounts Receivable Growth,18.30%,54.31%
"Accounts Receivables, Net",28.18B,2.95B
"Accounts Receivables, Gross",28.18B,2.95B
